In [3]:
from selenium import webdriver # 시간이 좀 오래걸림
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

dir = 'C:/Users/juwan/Desktop/산학프로젝트/'

In [2]:
# format으로 
next_jeongchak = ['#srchFrm > div.sch-result-wrap.compare-result-list > div.result-list-box > ul > li:nth-child(1) > a',
'#srchFrm > div.sch-result-wrap.compare-result-list > div.result-list-box > ul > li:nth-child(2) > a',
'#srchFrm > div.sch-result-wrap.compare-result-list > div.result-list-box > ul > li:nth-child(3) > a', 
'#srchFrm > div.sch-result-wrap.compare-result-list > div.result-list-box > ul > li:nth-child(4) > a',
'#srchFrm > div.sch-result-wrap.compare-result-list > div.result-list-box > ul > li:nth-child(5) > a',
'#srchFrm > div.sch-result-wrap.compare-result-list > div.result-list-box > ul > li:nth-child(6) > a',
'#srchFrm > div.sch-result-wrap.compare-result-list > div.result-list-box > ul > li:nth-child(7) > a',
'#srchFrm > div.sch-result-wrap.compare-result-list > div.result-list-box > ul > li:nth-child(8) > a',
'#srchFrm > div.sch-result-wrap.compare-result-list > div.result-list-box > ul > li:nth-child(9) > a',
'#srchFrm > div.sch-result-wrap.compare-result-list > div.result-list-box > ul > li:nth-child(10) > a',
'#srchFrm > div.sch-result-wrap.compare-result-list > div.result-list-box > ul > li:nth-child(11) > a',
'#srchFrm > div.sch-result-wrap.compare-result-list > div.result-list-box > ul > li:nth-child(12) > a']

In [205]:
# 최종코드 (한번에 하면 에러 - 20페이지씩 끊어서 수집)

young_list = []
young_name = []
young_dec = []
region = []

chrome_loc = 'C:/Users/juwan/Desktop/chromedriver.exe'

for i in range(1,20):
    browser = webdriver.Chrome(chrome_loc)
    
    url_target = 'https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifList.do?plcyTpOpenTy=list_004001&srchPlcyTp=004001&pageIndex={}&trgtJynEmp=&trgtJynEmp='.format(i)
    browser.get(url_target)
    
    young_html = browser.page_source
    soup = BeautifulSoup(young_html, 'html.parser')
    region.extend([i.text.strip() for i in soup.select('.badge')])
    
    for i in next_jeongchak: # 정책하나씩넘기기
        browser.find_element_by_css_selector(i).click() 
    
        young_html = browser.page_source
        soup = BeautifulSoup(young_html, 'html.parser')
        young_list.append(soup.select('.list_cont'))
        young_name.append("".join([i.text.strip() for i in soup.select('.plcy-left')]))
        young_dec.append("".join([i.text.strip() for i in soup.select('.bullet-arrow1')]))
        
        browser.find_element_by_css_selector('#content > div.btn_wrap.view-btn.green_type > a').click()

In [215]:
df_example = pd.DataFrame()

In [216]:
type = []
for i in young_list:
    type.append(i[0].text.strip())
    
content = []
for i in young_list:
    content.append(i[1].text.strip())
    
age = []
for i in young_list:
    age.append(i[5].text.strip())
    
region_income = []
for i in young_list:
    region_income.append(i[6].text.strip())
    
edu = []
for i in young_list:
    edu.append(i[7].text.strip())
    
major = []
for i in young_list:
    major.append(i[8].text.strip())

job = []
for i in young_list:
    job.append(i[9].text.strip())
    
special = []
for i in young_list:
    special.append(i[10].text.strip())
    
plus = []
for i in young_list:
    plus.append(i[11].text.strip())
    
limit = []
for i in young_list:
    limit.append(i[12].text.strip())

In [217]:
df_example['name'] = young_name    # 정책명
df_example['description'] = young_dec    # 정책 내용

df_example['type'] = type    # 정책 유형
df_example['content'] = content    # 지원 내용

df_example['age'] = age    # 연령
df_example['region_income'] = region_income    # 거주지 및 소득
df_example['edu'] = edu    # 학력
df_example['major'] = major    # 전공
df_example['job'] = job    # 취업 상태
df_example['special'] = special    # 특화 분야
df_example['plus'] = plus    # 추가 단서 사항
df_example['limit'] = limit    # 참여 제한 대상

df_example['region'] = region    # 정책 지자체
df_example['URL']  = 'https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifList.do'    # 온라인 청년센터 URL

In [219]:
df_example = df_example.reset_index()    # index 설정

In [224]:
dir = 'C:/Users/juwan/Desktop/산학프로젝트/'

# csv로 저장
# df_example.to_csv(dir + 'online_jeongchaek.csv', encoding = 'utf-8', index = False) # online_jeongchaek.csv

In [ ]:
# pickle로 저장

import pickle
 
# Save pickle 
# with open("online_jeongchaek.pickle","wb") as fw:
#     pickle.dump(df_example, fw)
 
 #  Load pickle|
with open("online_jeongchaek.pickle","rb") as fr:
    df_example = pickle.load(fr)

### 온라인청년센터 데이터에 age -> min_age/max_age로 나눔


In [4]:
df = pd.read_csv(dir + 'online_jeongchaek.csv')

In [6]:
#최소나이, 최대나이 col 생성

a = df['age'].str
df['max_age'] = a[-4:-1]
df['min_age'] = a[1:4]

In [8]:
replace_zero = df['min_age'].replace(' 0세', "0")
df['min_age'] = replace_zero

In [19]:
# 인덱스 붙임
df = df.reset_index()

In [20]:
# df.to_csv('age.csv', header=True, index=False)

### csv 파일 안에서 세, 이상, 이하 등 수정 -> age_fix.csv

In [ ]:
df = pd.read_csv('online_jeongchaek_fix.csv', encoding='cp949')

In [36]:
df['min_age'].unique()

array(['15', '18', '0', '제한없음', '19', '21', '20', '16', '1', '25', '9',
       '8'], dtype=object)

In [37]:
df['max_age'].unique()

array(['69', '34', '75', '제한없음', '39', '100', '35', '99', '40', '29',
       '49', '60', '55', '64', '33', '30', '24', '45', '26', '19', '50',
       '25', '20', '44', '65'], dtype=object)